# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse                Shipments  \
0  Jun 14 2023  4:44PM  264784        19                  8011026   
1  Jun 14 2023  4:43PM  264783        19              ADV80021106   
2  Jun 14 2023  4:43PM  264783        19              ADV80021107   
3  Jun 14 2023  4:40PM  264781        19  ADV80021109-Hold_Rx(MN)   
4  Jun 14 2023  4:40PM  264781        19  ADV80021110-Hold_Rx(MI)   
5  Jun 14 2023  4:40PM  264781        19              ADV80021111   
6  Jun 14 2023  4:09PM  264779        19                MSP222692   
7  Jun 14 2023  3:48PM  264777        12                  9573032   
8  Jun 14 2023  3:04PM  264768        10                  9573037   
9  Jun 14 2023  1:29PM  264758        10                 HH-42598   

              Customer ShipmentStatus  
0  AdvaGen Pharma, Ltd       Released  
1  AdvaGen Pharma, Ltd       Released  
2  AdvaGen Pharma, Ltd       Released  
3  AdvaGen Pharma, Ltd       Released  
4  AdvaGen Pharma, Ltd       Released  
5  AdvaGen Pharma, Ltd       Released  
6     Methapharm, Inc.       Released  
7     SD Head USA, LLC      Executing  
8              Bio-PRF       Released  
9            Hush Hush       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
12  264777      Executing          1
13  264779       Released          1
14  264781       Released          3
15  264783       Released          2
16  264784       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
264777                NaN        1.0       NaN
264779                NaN        NaN       1.0
264781                NaN        NaN       3.0
264783                NaN        NaN       2.0
264784                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
264707               16.0        3.0       0.0
264713                0.0        0.0       2.0
264726                0.0        0.0       3.0
264733                0.0        0.0      13.0
264740                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
264707                 16          3         0
264713                  0          0         2
264726                  0          0         3
264733                  0          0        13
264740                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               264707         16          3         0
1               264713          0          0         2
2               264726          0          0         3
3               264733          0          0        13
4               264740          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               264707        16         3         
1               264713                            2
2               264726                            3
3               264733                           13
4               264740                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                     Customer
0   Jun 14 2023  4:44PM  264784        19          AdvaGen Pharma, Ltd
1   Jun 14 2023  4:43PM  264783        19          AdvaGen Pharma, Ltd
3   Jun 14 2023  4:40PM  264781        19          AdvaGen Pharma, Ltd
6   Jun 14 2023  4:09PM  264779        19             Methapharm, Inc.
7   Jun 14 2023  3:48PM  264777        12             SD Head USA, LLC
8   Jun 14 2023  3:04PM  264768        10                      Bio-PRF
9   Jun 14 2023  1:29PM  264758        10                    Hush Hush
14  Jun 14 2023  1:29PM  264757        20  Reliable 1 Laboratories LLC
38  Jun 14 2023  1:28PM  264756        10          Digital Brands, LLC
40  Jun 14 2023 11:18AM  264746        21            NBTY Global, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                     Customer  \
0  Jun 14 2023  4:44PM  264784        19          AdvaGen Pharma, Ltd   
1  Jun 14 2023  4:43PM  264783        19          AdvaGen Pharma, Ltd   
2  Jun 14 2023  4:40PM  264781        19          AdvaGen Pharma, Ltd   
3  Jun 14 2023  4:09PM  264779        19             Methapharm, Inc.   
4  Jun 14 2023  3:48PM  264777        12             SD Head USA, LLC   
5  Jun 14 2023  3:04PM  264768        10                      Bio-PRF   
6  Jun 14 2023  1:29PM  264758        10                    Hush Hush   
7  Jun 14 2023  1:29PM  264757        20  Reliable 1 Laboratories LLC   
8  Jun 14 2023  1:28PM  264756        10          Digital Brands, LLC   
9  Jun 14 2023 11:18AM  264746        21            NBTY Global, Inc.   

  Completed Executing Released  
0                            1  
1                            2  
2                            3  
3                            1  
4                   1           
5                            1  
6                            5  
7                           24  
8                   1        1  
9                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                     Customer  \
0  Jun 14 2023  4:44PM  264784        19          AdvaGen Pharma, Ltd   
1  Jun 14 2023  4:43PM  264783        19          AdvaGen Pharma, Ltd   
2  Jun 14 2023  4:40PM  264781        19          AdvaGen Pharma, Ltd   
3  Jun 14 2023  4:09PM  264779        19             Methapharm, Inc.   
4  Jun 14 2023  3:48PM  264777        12             SD Head USA, LLC   
5  Jun 14 2023  3:04PM  264768        10                      Bio-PRF   
6  Jun 14 2023  1:29PM  264758        10                    Hush Hush   
7  Jun 14 2023  1:29PM  264757        20  Reliable 1 Laboratories LLC   
8  Jun 14 2023  1:28PM  264756        10          Digital Brands, LLC   
9  Jun 14 2023 11:18AM  264746        21            NBTY Global, Inc.   

  Released Executing Completed  
0        1                      
1        2                      
2        3                      
3        1                      
4                  1            
5        1                      
6        5                      
7       24                      
8        1         1            
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse             Customer Released  \
0  Jun 14 2023  4:44PM  264784        19  AdvaGen Pharma, Ltd        1   
1  Jun 14 2023  4:43PM  264783        19  AdvaGen Pharma, Ltd        2   
2  Jun 14 2023  4:40PM  264781        19  AdvaGen Pharma, Ltd        3   
3  Jun 14 2023  4:09PM  264779        19     Methapharm, Inc.        1   
4  Jun 14 2023  3:48PM  264777        12     SD Head USA, LLC            

  Executing Completed  
0                      
1                      
2                      
3                      
4         1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Jun 14 2023  4:44PM  264784        19  AdvaGen Pharma, Ltd       1.0   
1  Jun 14 2023  4:43PM  264783        19  AdvaGen Pharma, Ltd       2.0   
2  Jun 14 2023  4:40PM  264781        19  AdvaGen Pharma, Ltd       3.0   
3  Jun 14 2023  4:09PM  264779        19     Methapharm, Inc.       1.0   
4  Jun 14 2023  3:48PM  264777        12     SD Head USA, LLC       NaN   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        1.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Jun 14 2023  4:44PM  264784        19  AdvaGen Pharma, Ltd       1.0   
1  Jun 14 2023  4:43PM  264783        19  AdvaGen Pharma, Ltd       2.0   
2  Jun 14 2023  4:40PM  264781        19  AdvaGen Pharma, Ltd       3.0   
3  Jun 14 2023  4:09PM  264779        19     Methapharm, Inc.       1.0   
4  Jun 14 2023  3:48PM  264777        12     SD Head USA, LLC       0.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        1.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1059015      20.0        1.0        0.0
12          264777       0.0        1.0        0.0
19         1588574       8.0        3.0       16.0
20          794196      29.0        0.0        0.0
21          264746       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1059015      20.0        1.0        0.0
1        12   264777       0.0        1.0        0.0
2        19  1588574       8.0        3.0       16.0
3        20   794196      29.0        0.0        0.0
4        21   264746       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      20.0        1.0        0.0
1        12       0.0        1.0        0.0
2        19       8.0        3.0       16.0
3        20      29.0        0.0        0.0
4        21       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   20.0
1        12  Released    0.0
2        19  Released    8.0
3        20  Released   29.0
4        21  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    19    20   21
Status                               
Completed   0.0  0.0  16.0   0.0  0.0
Executing   1.0  1.0   3.0   0.0  0.0
Released   20.0  0.0   8.0  29.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    19    20   21
0          Completed   0.0  0.0  16.0   0.0  0.0
1          Executing   1.0  1.0   3.0   0.0  0.0
2           Released  20.0  0.0   8.0  29.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    19    20   21
0  Completed   0.0  0.0  16.0   0.0  0.0
1  Executing   1.0  1.0   3.0   0.0  0.0
2   Released  20.0  0.0   8.0  29.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()